In [ ]:
!pip install openai langchain langchain-community langchain-core PyPDF2 accelerate bitsandbytes llama_index huggingface_hub chromadb

In [ ]:
!pip install transformers sentence-transformers -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 4.1.0
    Uninstalling sentence-transformers-4.1.0:
      Successfully uninstalled sentence-transformers-4.1.0


In [ ]:
!pip install markitdown[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/

In [ ]:
import chromadb
chroma_client = chromadb.Client()

In [ ]:
collection = chroma_client.create_collection(name="my_collection")

UniqueConstraintError: Collection my_collection already exists

In [ ]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 96.9MiB/s]


In [ ]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)

{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.0404009819030762, 1.2430799007415771]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [ ]:
import chromadb
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection")

# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

results = collection.query(
    query_texts=["This is a query document about florida"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)


{'ids': [['id2', 'id1']], 'embeddings': None, 'documents': [['This is a document about oranges', 'This is a document about pineapple']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.1462137699127197, 1.3015384674072266]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


# JITC Integration with ChromaDB

In [ ]:
from google.colab import drive
drive.mount('/GD', force_remount=True)

Mounted at /GD


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Get all protocol standard files into chromadb

Each protocol standard sections file is further sliced into chunks of 512 tokens, and stored in chromadb as vectors with the file and chunk name in their metadata.



In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from PyPDF2 import PdfReader
import os

# Initialize ChromaDB client
#chroma_client = chromadb.Client(Settings(persist_directory="/GD/My Drive/JITC/chroma_db"))
chroma_client = chromadb.PersistentClient(path="/GD/My Drive/JITC/chroma_db")

# Load Sentence Transformer model (optimized for semantic tasks)
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

# Function to extract and chunk text from PDFs
def extract_and_chunk_text_from_pdf(pdf_path, max_chunk_size=512):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"

    # Split text into chunks based on token size
    sentences = text.split(". ")
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk.split()) + len(sentence.split()) <= max_chunk_size:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    if current_chunk:  # Add last chunk
        chunks.append(current_chunk.strip())
    return chunks

# Store PDFs with Sentence Transformer embeddings in ChromaDB
def store_pdfs_in_chromadb(folder_path, chroma_collection_name, distance_metric="cosine"):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]

    # Create a collection with the specified distance metric
    collection = chroma_client.get_or_create_collection(
        name=chroma_collection_name,
        metadata={"hnsw:space": distance_metric}
    )

    for pdf_file in pdf_files:
        # Extract and chunk text
        pdf_path = os.path.join(folder_path, pdf_file)
        chunks = extract_and_chunk_text_from_pdf(pdf_path)

        # Generate embeddings
        embeddings = embedding_model.encode(chunks).tolist()

        # Prepare metadata and IDs
        metadatas = [{"document_name": pdf_file} for _ in range(len(chunks))]
        ids = [f"{pdf_file}_chunk_{i}" for i in range(len(chunks))]

        # Insert into ChromaDB
        collection.upsert(documents=chunks, ids=ids, metadatas=metadatas, embeddings=embeddings)


    #collection.persist()

    print(f"Processed and stored {len(pdf_files)} PDFs into ChromaDB collection '{chroma_collection_name}' with distance metric '{distance_metric}'.")


# Main function
if __name__ == "__main__":
    folder_path = "/GD/My Drive/JITC"
    chroma_collection_name = "protocol_standards"
    store_pdfs_in_chromadb(folder_path, chroma_collection_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Processed and stored 59 PDFs into ChromaDB collection 'protocol_standards' with distance metric 'cosine'.


In [ ]:
collection = chroma_client.get_collection(name=chroma_collection_name)
print(f"Total chunks stored: {collection.count()}")

Total chunks stored: 229


In [ ]:
def list_all_chunks(chroma_collection_name):

    collection = chroma_client.get_collection(name=chroma_collection_name)


    results = collection.get(include=["metadatas"])

    # Display the chunk IDs and their metadata
    for doc_id, metadata in zip(results["ids"], results["metadatas"]):
        print(f"Chunk ID: {doc_id}")
        print(f"Metadata: {metadata}")
        print("-" * 80)

# Main function to view stored chunk list
if __name__ == "__main__":
    chroma_collection_name = "protocol_standards"
    list_all_chunks(chroma_collection_name)


Chunk ID: IPv6_part1.pdf_chunk_0
Metadata: {'document_name': 'IPv6_part1.pdf'}
--------------------------------------------------------------------------------
Chunk ID: IPv6_part1.pdf_chunk_1
Metadata: {'document_name': 'IPv6_part1.pdf'}
--------------------------------------------------------------------------------
Chunk ID: IPv6_part1.pdf_chunk_2
Metadata: {'document_name': 'IPv6_part1.pdf'}
--------------------------------------------------------------------------------
Chunk ID: IPv6_part1.pdf_chunk_3
Metadata: {'document_name': 'IPv6_part1.pdf'}
--------------------------------------------------------------------------------
Chunk ID: IPv6_part2.pdf_chunk_0
Metadata: {'document_name': 'IPv6_part2.pdf'}
--------------------------------------------------------------------------------
Chunk ID: IPv6_part2.pdf_chunk_1
Metadata: {'document_name': 'IPv6_part2.pdf'}
--------------------------------------------------------------------------------
Chunk ID: IPv6_part2.pdf_chunk_2
Metadat

In [ ]:
def count_chunks(chroma_collection_name):
    # Retrieve the collection
    collection = chroma_client.get_collection(name=chroma_collection_name)

    # Retrieve all stored data
    results = collection.get(include=["metadatas"])
    total_chunks = len(results["ids"])

    print(f"Total Chunks: {total_chunks}")
    return total_chunks


chroma_collection_name = "protocol_standards"
count_chunks(chroma_collection_name)


Total Chunks: 210


210

In [ ]:
def query_and_rank_chunks(chroma_collection_name, query_text, top_n=5):
    # Retrieve the collection
    collection = chroma_client.get_collection(name=chroma_collection_name)

    # Generate the embedding for the query text
    query_embedding = embedding_model.encode([query_text]).tolist()

    # Query the ChromaDB collection
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_n,
        include=["documents", "metadatas", "distances"]
    )

    # Display the top chunks and their scores
    for i, (doc, metadata, distance) in enumerate(zip(results["documents"][0], results["metadatas"][0], results["distances"][0])):
        print(f"Rank: {i + 1}")
        print(f"Chunk: {doc}")
        print(f"Metadata: {metadata}")
        print(f"Score (Distance): {distance}")
        print("-" * 80)

# Example usage
if __name__ == "__main__":
    chroma_collection_name = "protocol_standards"
    query_text = "Explain the Kerberos structure."
    query_and_rank_chunks(chroma_collection_name, query_text, top_n=5)

Rank: 1
Chunk: RFC 4121               Kerberos Version 5 GSS-API              July 2005
4.2.6.2.  Wrap Tokens
Use of the GSS_Wrap() call yields a token (referred as the Wrap token
in this document), which consists of a descriptive header, followed
by a body portion that contains either the input user data in
plaintext concatenated with the checksum, or the input user data
encrypted.  The GSS_Wrap() token SHALL have the following format:
Octet no   Name        Description
--------------------------------------------------------------
0..1     TOK_ID    Identification field.  Tokens emitted by
GSS_Wrap() contain the hex value 05 04
expressed in big-endian order in this
field.
2        Flags     Attributes field, as described in section
4.2.2.
3        Filler    Contains the hex value FF.
4..5     EC        Contains the "extra count" field, in big-
endian order as described in section 4.2.3.
6..7     RRC       Contains the "right rotation count" in big-
endian order, as described in secti

In [ ]:
def query_and_list_scores(chroma_collection_name, query_text, top_n=5):
    # Retrieve the collection
    collection = chroma_client.get_collection(name=chroma_collection_name)

    # Generate the embedding for the query text
    query_embedding = embedding_model.encode([query_text]).tolist()

    # Query the ChromaDB collection
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_n,
        include=["embeddings","metadatas", "distances"]
    )

    # Display the rank, metadata, and distance score for the top results
    for i, (doc_id, embedding, metadata, distance) in enumerate(zip(results["ids"][0], results["embeddings"][0], results["metadatas"][0], results["distances"][0])):
        print(f"Rank: {i + 1}")
        print(f"Document ID: {doc_id}")
        #print(f"Embeddings: {embedding}")
        print(f"Metadata: {metadata}")
        print(f"Distance Score: {distance}")
        print("-" * 80)
        print("-" * 80)

# Example usage
if __name__ == "__main__":
    chroma_collection_name = "protocol_standards"
    query_text = "Explain the Kerberos structure"
    query_and_list_scores(chroma_collection_name, query_text, top_n=5)


Rank: 1
Document ID: Kerberos_part4.pdf_chunk_0
Metadata: {'document_name': 'Kerberos_part4.pdf'}
Distance Score: 0.474065363407135
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Rank: 2
Document ID: Kerberos_part2.pdf_chunk_0
Metadata: {'document_name': 'Kerberos_part2.pdf'}
Distance Score: 0.4754754304885864
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Rank: 3
Document ID: Kerberos_part5.pdf_chunk_0
Metadata: {'document_name': 'Kerberos_part5.pdf'}
Distance Score: 0.47971177101135254
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Rank: 4
Document ID: Kerberos_part4.pdf_chunk_1
Metadata: {'document_name': 'Kerberos_part4.pdf'}
Distance Score: 0

# Load Chromadb Database

In [ ]:
!pip install chromadb

In [ ]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

def load_chromadb(path):
    """
    Load the ChromaDB database from the specified persistent directory.

    Args:
        persist_directory (str): Path to the persistent directory where ChromaDB data is stored.

    Returns:
        chromadb.Client: The ChromaDB client with the loaded database.
    """
    # Initialize the ChromaDB client with the persistence directory
    #client = chromadb.Client(Settings(persist_directory=persist_directory))
    chroma_client = chromadb.PersistentClient(path="/GD/My Drive/JITC/chroma_db")
    return chroma_client

# Example usage
if __name__ == "__main__":
    persist_directory = "/GD/My Drive/JITC/chroma_db"  # Path to persistent directory
    chroma_client = load_chromadb(persist_directory)

    # List collections to confirm the database is loaded
    collections = chroma_client.list_collections()
    print("Loaded Collections:")
    for collection in collections:
        print(collection)

OSError: [Errno 107] Transport endpoint is not connected: '/GD/My Drive'

In [ ]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import os

def load_chromadb(path):
    """
    Load the ChromaDB database from the specified persistent directory.

    Args:
        persist_directory (str): Path to the persistent directory where ChromaDB data is stored.

    Returns:
        chromadb.Client: The ChromaDB client with the loaded database.
    """
    # Ensure the path exists and is a directory
    if os.path.exists(path) and not os.path.isdir(path):
        # If it's not a directory, rename it (or delete, depending on your needs)
        os.rename(path, path + ".old") # Rename to avoid data loss
        print(f"Renamed existing file at {path} to {path}.old")

    # If it doesn't exist or was renamed, create the directory
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
        print(f"Created directory at {path}")

    # Initialize the ChromaDB client with the persistence directory
    chroma_client = chromadb.PersistentClient(path=path)
    return chroma_client

# Example usage
if __name__ == "__main__":
    persist_directory = "/GD/My Drive/JITC/chroma_db/"  # Path to persistent directory
    chroma_client = load_chromadb(persist_directory)

    # List collections to confirm the database is loaded
    collections = chroma_client.list_collections()
    print("Loaded Collections:")
    for collection in collections:
        print(collection)

Loaded Collections:
protocol_standards


In [ ]:
def query_and_list_scores(chroma_collection_name, query_text, top_n=5):
    # Retrieve the collection
    collection = chroma_client.get_collection(name=chroma_collection_name)

    embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
    # Generate the embedding for the query text
    query_embedding = embedding_model.encode([query_text]).tolist()

    # Query the ChromaDB collection
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_n,
        include=["embeddings","metadatas", "distances"]
    )

    # Display the rank, metadata, and distance score for the top results
    for i, (doc_id, embedding, metadata, distance) in enumerate(zip(results["ids"][0], results["embeddings"][0], results["metadatas"][0], results["distances"][0])):
        print(f"Rank: {i + 1}")
        print(f"Document ID: {doc_id}")
        #print(f"Embeddings: {embedding}")
        print(f"Metadata: {metadata}")
        print(f"Distance Score: {distance}")
        print("-" * 80)
        print("-" * 80)

# Example usage
if __name__ == "__main__":
    chroma_collection_name = "protocol_standards"
    query_text = "Explain the IPv6 structure"
    query_and_list_scores(chroma_collection_name, query_text, top_n=5)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Rank: 1
Document ID: IPv6_part1.pdf_chunk_0
Metadata: {'document_name': 'IPv6_part1.pdf'}
Distance Score: 0.325980007648468
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Rank: 2
Document ID: IPv6_part2.pdf_chunk_0
Metadata: {'document_name': 'IPv6_part2.pdf'}
Distance Score: 0.325980007648468
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Rank: 3
Document ID: IPv6_part5.pdf_chunk_0
Metadata: {'document_name': 'IPv6_part5.pdf'}
Distance Score: 0.3738921284675598
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Rank: 4
Document ID: IPv6_part3.pdf_chunk_2
Metadata: {'document_name': 'IPv6_part3.pdf'}
Distance Score: 0.38138246536254883
---------------

# Military Standard 188 for Test Plan

In [ ]:
import os
import json
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from markitdown import MarkItDown
from openai import OpenAI
from google.colab import userdata
import chromadb
from PIL import Image

# ==== ChromaDB Setup ====
chroma_client = chromadb.PersistentClient(path="/GD/My Drive/JITC/military_standards")

# ==== LLM Setup for MarkItDown ====
os.environ["OPENAI_API_KEY"] = userdata.get('OpenaAIAPI')
client = OpenAI()
md = MarkItDown(llm_client=client, llm_model="o4-mini")

# ==== Embedding Model ====
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

# ==== Extract Text and Images Per Page ====
def extract_text_and_images_from_pdf(pdf_path, image_output_folder):
    reader = PdfReader(pdf_path)
    pages_data = []
    os.makedirs(image_output_folder, exist_ok=True)
    print(f"Extracting from {pdf_path}")
    for i, page in enumerate(reader.pages):
        text = page.extract_text() or ""
        resources = page.get("/Resources")
        image_files = []
        if resources and "/XObject" in resources:
            xobjects = resources["/XObject"].get_object()
            for xobj_name in xobjects:
                xobj = xobjects[xobj_name]
                if xobj.get("/Subtype") == "/Image":
                    try:
                        filters = xobj.get('/Filter')
                        print(f"Page {i+1}: Found image {xobj_name}, filters: {filters}")
                        data = xobj.get_data()
                        img_ext = 'jpg' if filters == '/DCTDecode' else 'png'
                        base_name = os.path.basename(pdf_path).replace("/", "_")
                        img_filename = f"{image_output_folder}/{base_name}_page_{i+1}_{xobj_name[1:]}.{img_ext}"
                        with open(img_filename, "wb") as img_file:
                            img_file.write(data)
                        # Normalize PNG images so MarkItDown/OpenAI Vision can handle them
                        if img_ext == 'png':
                            try:
                                with Image.open(img_filename) as im:
                                    # Convert to grayscale ("L") if needed
                                    if im.mode not in ("L", "RGB"):
                                        im = im.convert("L")
                                    im.save(img_filename)
                            except Exception as e:
                                print(f"Image conversion failed for {img_filename}: {e}")
                        image_files.append(img_filename)
                        print(f"Saved image: {img_filename}")
                    except Exception as e:
                        print(f"Failed to extract image {xobj_name} on page {i+1}: {e}")
        pages_data.append({"page": i+1, "text": text, "images": image_files})
    return pages_data

# ==== Describe Images using MarkItDown ====
def describe_images_for_pages(pages_data):
    for page_info in pages_data:
        descriptions = []
        for img_path in page_info['images']:
            try:
                result = md.convert(img_path)
                desc = result.text_content if hasattr(result, 'text_content') else str(result)
            except Exception as e:
                desc = f"Image description failed: {e}"
            descriptions.append(desc)
        page_info['image_descriptions'] = descriptions
    return pages_data

# ==== Combine Text and Image Descriptions ====
def combine_page_content(page_info):
    text = page_info['text'].strip()
    image_descs = "\n".join(
        f"[Image {idx+1}]: {desc}"
        for idx, desc in enumerate(page_info.get('image_descriptions', []))
    )
    combined = text
    if image_descs:
        combined += "\n" + image_descs
    return combined.strip()

# ==== Main Storage Function ====
def store_pdfs_in_chromadb_with_images(folder_path, chroma_collection_name, distance_metric="cosine"):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    image_output_folder = os.path.join(folder_path, "pdf_images")
    os.makedirs(image_output_folder, exist_ok=True)

    collection = chroma_client.get_or_create_collection(
        name=chroma_collection_name,
        metadata={"hnsw:space": distance_metric}
    )

    doc_count = 0
    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        pages_data = extract_text_and_images_from_pdf(pdf_path, image_output_folder)
        pages_data = describe_images_for_pages(pages_data)

        # For each page, combine text and image descs, embed, and upsert
        for page_info in pages_data:
            combined_content = combine_page_content(page_info)
            if not combined_content.strip():
                continue  # skip empty pages

            embedding = embedding_model.encode([combined_content])[0].tolist()
            # Store lists as JSON strings for ChromaDB compatibility
            metadata = {
                "document_name": str(pdf_file),
                "page": int(page_info['page']),
                "image_files": json.dumps(page_info['images']),
                "image_descriptions": json.dumps(page_info.get('image_descriptions', []))
            }
            doc_id = f"{pdf_file}_page_{page_info['page']}"
            collection.upsert(
                documents=[combined_content],
                ids=[doc_id],
                metadatas=[metadata],
                embeddings=[embedding]
            )
            doc_count += 1

    print(f"Processed and stored {doc_count} pages (text + images) from {len(pdf_files)} PDFs into ChromaDB collection '{chroma_collection_name}'.")

# ==== Main Entrypoint ====
if __name__ == "__main__":
    folder_path = "/GD/My Drive/JITC/military_standards_pdf"
    chroma_collection_name = "military_standards"
    store_pdfs_in_chromadb_with_images(folder_path, chroma_collection_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Extracting from /GD/My Drive/JITC/military_standards_pdf/MIL-STD-188-203-1A_1995.pdf
Page 1: Found image /im1, filters: /CCITTFaxDecode
Saved image: /GD/My Drive/JITC/military_standards_pdf/pdf_images/MIL-STD-188-203-1A_1995.pdf_page_1_im1.png
Page 20: Found image /im2, filters: /CCITTFaxDecode
Saved image: /GD/My Drive/JITC/military_standards_pdf/pdf_images/MIL-STD-188-203-1A_1995.pdf_page_20_im2.png
Page 23: Found image /im3, filters: /CCITTFaxDecode
Saved image: /GD/My Drive/JITC/military_standards_pdf/pdf_images/MIL-STD-188-203-1A_1995.pdf_page_23_im3.png
Page 24: Found image /im4, filters: /CCITTFaxDecode
Saved image: /GD/My Drive/JITC/military_standards_pdf/pdf_images/MIL-STD-188-203-1A_1995.pdf_page_24_im4.png
Page 25: Found image /im5, filters: /CCITTFaxDecode
Saved image: /GD/My Drive/JITC/military_standards_pdf/pdf_images/MIL-STD-188-203-1A_1995.pdf_page_25_im5.png
Page 26: Found image /im6, filters: /CCITTFaxDecode
Saved image: /GD/My Drive/JITC/military_standards_pdf/pdf_im

In [ ]:
import json

def print_all_chunks_with_images(chroma_collection_name, max_results=5000):
    collection = chroma_client.get_collection(name=chroma_collection_name)
    # Do NOT include "ids" in the include list
    results = collection.get(
        include=["documents", "metadatas"],  # Corrected!
        limit=max_results
    )

    count = 0
    for doc_id, document, metadata in zip(
            results["ids"], results["documents"], results["metadatas"]):
        has_images = False
        image_files = []
        # Try to decode image_files if present
        if "image_files" in metadata:
            try:
                image_files = json.loads(metadata["image_files"])
                if isinstance(image_files, list) and len(image_files) > 0:
                    has_images = True
            except Exception:
                pass
        if has_images:
            count += 1
            print(f"\n--- Chunk with Images ---")
            print(f"Document ID: {doc_id}")
            print(f"Page: {metadata.get('page')}")
            print(f"Document Name: {metadata.get('document_name')}")
            print(f"Image Files: {image_files}")
            # Image descriptions
            if "image_descriptions" in metadata:
                try:
                    descs = json.loads(metadata["image_descriptions"])
                    print("Image Descriptions:")
                    for idx, desc in enumerate(descs):
                        print(f"  Image {idx+1}: {desc}")
                except Exception:
                    print(f"Image Descriptions: {metadata['image_descriptions']}")
            print("Full Document Text (truncated to 500 chars):")
            print(document[:500] + ("..." if len(document) > 500 else ""))
            print("=" * 80)
    if count == 0:
        print("No chunks with images found.")

# Example usage:
if __name__ == "__main__":
    chroma_collection_name = "military_standards"
    print_all_chunks_with_images(chroma_collection_name)



--- Chunk with Images ---
Document ID: MIL-STD-188-203-1A_1995.pdf_page_1
Page: 1
Document Name: MIL-STD-188-203-1A_1995.pdf
Image Files: ['/GD/My Drive/JITC/military_standards_pdf/pdf_images/MIL-STD-188-203-1A_1995.pdf_page_1_im1.png']
Image Descriptions:
  Image 1: 
# Description:
A monochrome, circular seal reading “DEPARTMENT OF DEFENSE” along its upper arc and “UNITED STATES OF AMERICA” along its lower arc.  Within the outer ring sits a broad-winged American bald eagle facing forward.  The eagle’s talons clutch a sheaf of arrows (symbolizing military readiness), and its outstretched wings nearly span the seal’s diameter.  Above the eagle, a semicircle of thirteen stars—each representing one of the original states—rises in a fan of straight rays.  The eagle is framed below by a laurel wreath, signifying honor and achievement.  The overall design conveys authority, unity, and the defensive mission of the U.S. Department of Defense.

Full Document Text (truncated to 500 chars):
MIL-

In [ ]:
import json
from sentence_transformers import SentenceTransformer

def query_and_list_scores(chroma_collection_name, query_text, top_n=5):
    # Retrieve the collection
    collection = chroma_client.get_collection(name=chroma_collection_name)

    embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
    # Generate the embedding for the query text
    query_embedding = embedding_model.encode([query_text]).tolist()

    # Query the ChromaDB collection
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_n,
        include=["documents", "embeddings", "metadatas", "distances"]
    )

    # Display the rank, metadata, and distance score for the top results
    for i, (doc_id, document, metadata, distance) in enumerate(
            zip(results["ids"][0], results["documents"][0], results["metadatas"][0], results["distances"][0])):
        print(f"\nRank: {i + 1}")
        print(f"Document ID: {doc_id}")
        print(f"Page: {metadata.get('page')}")
        print(f"Document Name: {metadata.get('document_name')}")
        # Decode image_files and image_descriptions if present and in JSON format
        if 'image_files' in metadata:
            try:
                image_files = json.loads(metadata['image_files'])
                print(f"Image Files: {image_files}")
            except Exception:
                print(f"Image Files: {metadata['image_files']}")
        if 'image_descriptions' in metadata:
            try:
                image_descriptions = json.loads(metadata['image_descriptions'])
                print("Image Descriptions:")
                for idx, desc in enumerate(image_descriptions):
                    print(f"  Image {idx+1}: {desc}")
            except Exception:
                print(f"Image Descriptions: {metadata['image_descriptions']}")
        print(f"Distance Score: {distance}")
        print("-" * 80)
        print("Top Document Text (truncated):")
        print(document[:500] + ("..." if len(document) > 500 else ""))
        print("=" * 80)

# Example usage
if __name__ == "__main__":
    chroma_collection_name = "military_standards"
    query_text = "Explain the TADIL A structure"
    query_and_list_scores(chroma_collection_name, query_text, top_n=5)



Rank: 1
Document ID: MIL-STD-188-203-1A_1995.pdf_page_67
Page: 67
Document Name: MIL-STD-188-203-1A_1995.pdf
Image Files: ['/GD/My Drive/JITC/military_standards_pdf/pdf_images/MIL-STD-188-203-1A_1995.pdf_page_67_im21.png']
Image Descriptions:
  Image 1: 
# Description:
Figure 10. TADIL-A System Block Diagram
This schematic shows two identical ground stations (left and right) each consisting of:
  1. TDS Computer – the host data terminal system
  2. Optional Security Equipment – used when TRANSEC/COMSEC encryption is required
  3. Data Terminal Set (DTS) – interfaces between the TDS computer and the radio network

From the DTS each station’s data stream is routed by a manual or automatic transfer switch into one of two tactical radio nets:

• HF Radio Net (upper path)
    – HF Transmitter/Receiver pair
    – Three serial RF Distribution nodes (to provide power-splitting, antenna switching and cabling to a constellation of low-data‐rate HF subsystems)
    – HF Receiver/Transmitter pair 